# 0. Cash Flow and Report

## 0.1 Problem

Daily cash flow and summary report 

## 0.2 Planning

- **Main objective**: summary report from the stock market negotiation of the day ( email), cash flow
                      - Python 3.0

# 1. Imports

In [3]:
import pandas as pd
import numpy as np
import datetime

import re

import smtplib

from email.message import EmailMessage

## 0.1. Helper Functions

In [4]:
# capture email and password from sender

def capture(arquivo): 
    nome_arq = arquivo
    try:
        arq = open(nome_arq, 'r') # abre arquivo em modo de leitura
        conteudo = arq.readlines() # lê as linhas do aquivo e guarda em uma lista
        arq.close()

        for linha in conteudo:
            if "email" in linha:
                lista_str = linha.split('=')
                email   = lista_str[1].strip()
                

            if "senha" in linha:
                lista_str = linha.split('=')
                senha     = lista_str[1].strip()
                              
    
    except FileNotFoundError:
        print(f"Arquivo {nome_arq} não encontrado!")
        
    return email, senha

In [5]:
def files(arq, name ): 
    
    add_file= str(input('Add another file to {}´s report ( only excel, please!)? ( N to exit )'.format(name)))

    regex = '(.xls|.xlsx)$' 


    spair_letter = [l for l in add_file   ] 
    

    if len(spair_letter) < 4:
            add_file = add_file.upper()[0]
            
    else:
        
        if add_file != "N":

            try: 
                if bool(re.search( regex, add_file )) == True  and add_file not in arq: 
                    
                    arq.append(add_file)
                else: 
                    print('Please check the file path and name. ej:("./reports/file.xlsx") ')
                    
                    
            except Exception as e:
                print('Error! : ',e)
        
            
    
    return arq



    

In [6]:
def send_mail_with_excel( from_email_file, name, recipient_email, subject, files_xls):
    #capture email and password "FROM"
    email, senha = capture(from_email_file)
    
    msg = EmailMessage()
    msg['Subject'] = subject + y
    msg['From'] = email
    msg['To'] = recipient_email
    
    email_content = f'''

        <p>Dear {name}, </p>
        <p>Please find attached detailed cash flow and trade report.  </p>
        <br />
        <p> <small><strong> Yesterday´s trade´s information   </strong></small> </p>
            {yesterday_report.to_html()}
        <br />
        <p>Taiz Alves</p>
        <p><small><strong>Data analyst </strong></small></p> '''

    msg.add_header('Content-Type', 'text/html')
    msg.set_payload(email_content)
    
    
    ## add more than 1 file and ask 
    
    for excel_file in files_xls:
        with open(excel_file, 'rb') as f:
            file_data = f.read()
            msg.add_attachment(file_data, maintype="application", subtype="xlsx", filename=excel_file)
    
    
    try: 
        with smtplib.SMTP_SSL('smtp.gmail.com', 465) as smtp:
            smtp.login(email, senha)
            smtp.sendmail(msg['From'], [msg['To']],msg.as_string().encode('utf-8'))
            smtp.quit()
            print('Email sent to {}'.format(recipient_email))
    except Exception as e:
        print('Error! : ',e)
        
    
        


# 2. Load data

In [7]:
data = pd.read_excel('../database/ReportHistory.xlsx', header=5, engine='openpyxl')
data.head()

,Posições,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,Horário,Position,Ativo,Tipo,Volume,Preço In,S / L,T / P,Horário Out,Preço Out,Comissão,Swap,Lucro,NaN,NaN,NaN
1,2020.10.01 14:00:00,477192759,WDOX20,buy,1,5645.5,5609,6045.5,2020.10.01 16:36:19,5661,0,0,311,NaN,NaN,NaN
2,2020.10.01 16:36:19,477549686,WDOX20,buy,1,5661,5620.5,6060.5,2020.10.01 17:47:59,5662,0,0,1914,NaN,NaN,NaN
3,2020.10.02 12:00:00,478175765,WDOX20,buy,1,5643,5605,6043,2020.10.02 12:15:44,5658,0,0,221,NaN,NaN,NaN
4,2020.10.02 12:15:44,478217920,WDOX20,buy,1,5658,5617.5,6057.5,2020.10.02 12:42:43,5672.5,0,0,1097,NaN,NaN,NaN


# 3.0 Data description

## 3.1 Rename columns

In [8]:
df1 = data.copy()

In [9]:

df1.rename(columns=df1.iloc[0], inplace= True)
df1 = df1.drop(labels=0, axis=0)
df1.head()


,Horário,Position,Ativo,Tipo,Volume,Preço In,S / L,T / P,Horário Out,Preço Out,Comissão,Swap,Lucro,NaN,NaN,NaN
1,2020.10.01 14:00:00,477192759,WDOX20,buy,1,5645.5,5609,6045.5,2020.10.01 16:36:19,5661,0,0,311,NaN,NaN,NaN
2,2020.10.01 16:36:19,477549686,WDOX20,buy,1,5661,5620.5,6060.5,2020.10.01 17:47:59,5662,0,0,1914,NaN,NaN,NaN
3,2020.10.02 12:00:00,478175765,WDOX20,buy,1,5643,5605,6043,2020.10.02 12:15:44,5658,0,0,221,NaN,NaN,NaN
4,2020.10.02 12:15:44,478217920,WDOX20,buy,1,5658,5617.5,6057.5,2020.10.02 12:42:43,5672.5,0,0,1097,NaN,NaN,NaN
5,2020.10.02 12:44:59,478327168,WDOX20,buy,1,5669.5,5629.5,6069.5,2020.10.02 12:54:47,5684,0,0,106,NaN,NaN,NaN


In [10]:
df1 = df1[['Horário',    'Position',       'Ativo',        'Tipo',
            'Volume',    'Preço In',       'S / L',       'T / P',
       'Horário Out',   'Preço Out', 'Lucro']]
df1.head()

,Horário,Position,Ativo,Tipo,Volume,Preço In,S / L,T / P,Horário Out,Preço Out,Lucro
1,2020.10.01 14:00:00,477192759,WDOX20,buy,1,5645.5,5609,6045.5,2020.10.01 16:36:19,5661,311
2,2020.10.01 16:36:19,477549686,WDOX20,buy,1,5661,5620.5,6060.5,2020.10.01 17:47:59,5662,1914
3,2020.10.02 12:00:00,478175765,WDOX20,buy,1,5643,5605,6043,2020.10.02 12:15:44,5658,221
4,2020.10.02 12:15:44,478217920,WDOX20,buy,1,5658,5617.5,6057.5,2020.10.02 12:42:43,5672.5,1097
5,2020.10.02 12:44:59,478327168,WDOX20,buy,1,5669.5,5629.5,6069.5,2020.10.02 12:54:47,5684,106


## 3.2  Data Dimention

In [11]:
print( 'Number of Rows: {}'.format( df1.shape[0] ) )
print( 'Number of Cols: {}'.format( df1.shape[1] ) )

Number of Rows: 332
Number of Cols: 11


## 3.3  Data Types

In [12]:
df1.dtypes

Horário        object
Position       object
Ativo          object
Tipo           object
Volume         object
Preço In       object
S / L          object
T / P          object
Horário Out    object
Preço Out      object
Lucro          object
dtype: object

In [13]:
df1['Horário'] = pd.to_datetime(df1['Horário'], format= '%Y-%m-%d ')
df1['Horário Out'] = pd.to_datetime(df1['Horário Out'], format= '%Y-%m-%d ')

df1['Position'] = df1['Position'].astype( np.int64)
df1['Preço In'] = df1['Preço In'].astype( float )
df1['Preço Out'] = df1['Preço Out'].astype( float )

df1['Lucro'] = df1['Lucro'].astype( np.int64)

df1.head()

,Horário,Position,Ativo,Tipo,Volume,Preço In,S / L,T / P,Horário Out,Preço Out,Lucro
1,2020-10-01 14:00:00,477192759,WDOX20,buy,1,5645.5,5609,6045.5,2020-10-01 16:36:19,5661.0,311
2,2020-10-01 16:36:19,477549686,WDOX20,buy,1,5661.0,5620.5,6060.5,2020-10-01 17:47:59,5662.0,1914
3,2020-10-02 12:00:00,478175765,WDOX20,buy,1,5643.0,5605,6043,2020-10-02 12:15:44,5658.0,221
4,2020-10-02 12:15:44,478217920,WDOX20,buy,1,5658.0,5617.5,6057.5,2020-10-02 12:42:43,5672.5,1097
5,2020-10-02 12:44:59,478327168,WDOX20,buy,1,5669.5,5629.5,6069.5,2020-10-02 12:54:47,5684.0,106


## 3.4 Feature extraction

In [14]:
df1['Date'] = df1['Horário'].dt.normalize()
df1['year'] =  df1['Horário'].dt.year
df1['month'] = df1['Horário'].dt.month
df1['day'] =   df1['Horário'].dt.day

In [15]:
df1['Ativo'].unique()

array(['WDOX20', 'WINV20', 'WINZ20', 'WDOZ20', 'WDOF21', 'CSNAF500',
       'WINA20', 'WDOA21', 'VALE3', 'PETR4', 'PETROZ29', 'ITUB4'],
      dtype=object)

In [16]:
# Market classification

df1['Market']= df1['Ativo'].apply(lambda x: 'futures' if x.startswith(('WIN', 'WDO')) else
                   
                   'stock' if len(x) == 5 else 'options' )


# day trade = 1 if day trade, 0 if not
df1['day_trade']= df1.apply(lambda x: 1 if x['day'] == x['Horário Out'].day else 0, axis=1 )
df1.tail()

,Horário,Position,Ativo,Tipo,Volume,Preço In,S / L,T / P,Horário Out,Preço Out,Lucro,Date,year,month,day,Market,day_trade
328,2021-03-25 12:14:42,577321190,CSNAF500,buy,5,NaN,NaN,NaN,2021-03-25 12:14:42,NaN,973,2021-03-25,2021,3,25,options,1
329,2021-03-25 15:14:43,570669884,CSNAF500,buy,7,NaN,NaN,NaN,2021-03-25 15:14:43,NaN,1784,2021-03-25,2021,3,25,options,1
330,2021-03-27 11:10:44,581764428,PETROZ29,sell,8,NaN,NaN,NaN,2021-04-27 11:10:44,NaN,1944,2021-03-27,2021,3,27,options,1
331,2021-03-28 15:14:45,544070573,CSNAF500,sell,1,NaN,NaN,NaN,2021-03-28 15:14:45,NaN,584,2021-03-28,2021,3,28,options,1
332,2021-03-29 15:14:46,543421108,CSNAF500,buy,7,NaN,NaN,NaN,2021-03-31 15:14:46,NaN,159,2021-03-29,2021,3,29,options,0


In [17]:
df1.isna().sum()

Horário         0
Position        0
Ativo           0
Tipo            0
Volume          0
Preço In       38
S / L          42
T / P          45
Horário Out     0
Preço Out      38
Lucro           0
Date            0
year            0
month           0
day             0
Market          0
day_trade       0
dtype: int64

## 3.4  Filtering Variables

In [18]:
summary = df1.groupby(['Date', 'Ativo'], as_index=False)['Lucro'].sum()
summary.head()

,Date,Ativo,Lucro
0,2020-10-01,WDOX20,2225
1,2020-10-02,WDOX20,4427
2,2020-10-05,WDOX20,10813
3,2020-10-06,WDOX20,7603
4,2020-10-07,WDOX20,7083


In [19]:
df1['Date'].max()

Timestamp('2021-03-29 00:00:00')

In [20]:

today = df1['Date'].max()

df1['yesterday'] = today - datetime.timedelta(days=1)

yesterday_report = df1.loc[df1['Date'] == df1['yesterday'],['Horário',    'Position',       'Ativo',        'Tipo',
            'Volume',  'Lucro', 'Market', 'day_trade']]


#y =datetime.date.today() - datetime.timedelta(days=1)
y =today - datetime.timedelta(days=1)
y = y.strftime('%Y_%m_%d')

## Save Report
#yesterday_report.to_excel('../reports/report{}.xls'.format(y))
yesterday_report

,Horário,Position,Ativo,Tipo,Volume,Lucro,Market,day_trade
331,2021-03-28 15:14:45,544070573,CSNAF500,sell,1,584,options,1


In [21]:
# Group by Date nr of trades and sum of Profif/loss
profit_loss = df1[['Date', 'Lucro']].groupby('Date').sum().reset_index()

trade_nr = df1[['Date', 'Ativo']].groupby('Date').count().reset_index()

#merge 
aux = pd.merge( trade_nr,profit_loss, how='inner', on='Date')
aux.columns =['Date', 'Nr of Trades', 'Profit/Loss']


summary_10_last_days = aux.sort_values('Date', ascending = False).head(10)
summary_10_last_days

,Date,Nr of Trades,Profit/Loss
56,2021-03-29,1,159
55,2021-03-28,1,584
54,2021-03-27,1,1944
53,2021-03-25,2,2757
52,2021-03-23,2,1442
51,2021-03-13,3,-379
50,2021-03-09,1,1705
49,2021-03-07,1,-333
48,2021-03-06,1,1245
47,2021-03-05,1,871


In [22]:
#Summarize stock, deal and total profit/loss per date using agg. 
total_report = df1.groupby(['Date', 'Ativo']).agg( Deals=('Ativo','count'),Profit_Loss=('Lucro','sum')
                                            ).sort_values('Date', ascending= False)

#total_report.to_excel('../reports/total_report_summary{}.xls'.format(y))
total_report.head(10)

Deals  Profit_Loss
Date       Ativo                       
2021-03-29 CSNAF500      1          159
2021-03-28 CSNAF500      1          584
2021-03-27 PETROZ29      1         1944
2021-03-25 CSNAF500      2         2757
2021-03-23 PETROZ29      1          577
           PETR4         1          865
2021-03-13 PETR4         1         -169
           ITUB4         1         -378
           WDOA21        1          168
2021-03-09 ITUB4         1         1705

In [23]:
daily_spend = df1.groupby('Date').agg({'Lucro':['count','sum']}).reset_index()
daily_spend= daily_spend.rename(columns={"count": "Nr. Trades", "sum": "Profit/Loss"})
daily_spend

Date      Lucro            
              Nr. Trades Profit/Loss
0  2020-10-01          2        2225
1  2020-10-02          5        4427
2  2020-10-05          8       10813
3  2020-10-06          9        7603
4  2020-10-07          6        7083
5  2020-10-13          5        3864
6  2020-10-14         16       19233
7  2020-10-19          3        2783
8  2020-10-20          6        7393
9  2020-10-21          4         -40
10 2020-10-22          3        6433
11 2020-10-23          5        6670
12 2020-10-26          1         296
13 2020-10-27          8        9820
14 2020-10-28         10       11087
15 2020-10-29         13        5507
16 2020-11-03         17       11792
17 2020-11-05          1        1152
18 2020-11-10         15       15779
19 2020-11-11          9        9953
20 2020-11-12         28       28200
21 2020-11-13          7        3302
22 2020-11-16         14        9724
23 2020-11-17         18       20036
24 2020-11-25         10       12016
25 2020-12-01          1         518
26 2020-12-02         13       15110
27 2020-12-03         16       12944
28 2020-12-07         13       11334
29 2020-12-08          2        -653
30 2020-12-09         14        9760
31 2020-12-10          3        2880
32 2020-12-11          9        6102
33 2021-02-11          2         599
34 2021-02-12          1         210
35 2021-02-13          1        1139
36 2021-02-14          1          85
37 2021-02-20          1        1074
38 2021-02-21          2         978
39 2021-02-23          1        2148
40 2021-02-25          2        2462
41 2021-02-26          1         465
42 2021-02-27          1        1278
43 2021-03-01          1         940
44 2021-03-02          1        -241
45 2021-03-03          8        5604
46 2021-03-04          1         953
47 2021-03-05          1         871
48 2021-03-06          1        1245
49 2021-03-07          1        -333
50 2021-03-09          1        1705
51 2021-03-13          3        -379
52 2021-03-23          2        1442
53 2021-03-25          2        2757
54 2021-03-27          1        1944
55 2021-03-28          1         584
56 2021-03-29          1         159

In [24]:
daily_spend['Lucro']['Profit/Loss'].values

array([ 2225,  4427, 10813,  7603,  7083,  3864, 19233,  2783,  7393,
         -40,  6433,  6670,   296,  9820, 11087,  5507, 11792,  1152,
       15779,  9953, 28200,  3302,  9724, 20036, 12016,   518, 15110,
       12944, 11334,  -653,  9760,  2880,  6102,   599,   210,  1139,
          85,  1074,   978,  2148,  2462,   465,  1278,   940,  -241,
        5604,   953,   871,  1245,  -333,  1705,  -379,  1442,  2757,
        1944,   584,   159], dtype=int64)

# 4.0 Cash Flow

In [25]:
c_flow = pd.DataFrame()


In [26]:
initial = 10000

initial_balance=[]
initial_balance.append(initial)


#c_flow.values
lucro = daily_spend['Lucro']['Profit/Loss'].values
tax = abs(lucro*0.01)

for i in range(daily_spend.shape[0] -1):
    initial = initial + lucro[i] - tax[i]
    initial_balance.append(initial)
    
#initial_balance
c_flow['Date'] = daily_spend['Date']
c_flow['initial_balance'] = initial_balance
c_flow['Lucro'] = daily_spend['Lucro']['Profit/Loss']
c_flow['withholding_tax']= c_flow['Lucro'].apply(lambda x: abs(0.01*x) )
c_flow['final_balance'] = c_flow['initial_balance'] + c_flow['Lucro']- c_flow['withholding_tax']

c_flow.sort_values('Date', ascending = False)

# Export 
#c_flow.sort_values('Date', ascending = False).to_excel('../reports/cashflow{}.xls'.format(y))


,Date,initial_balance,Lucro,withholding_tax,final_balance
56,2021-03-29,309616.32,159,1.59,309773.73
55,2021-03-28,309038.16,584,5.84,309616.32
54,2021-03-27,307113.60,1944,19.44,309038.16
53,2021-03-25,304384.17,2757,27.57,307113.60
52,2021-03-23,302956.59,1442,14.42,304384.17
51,2021-03-13,303339.38,-379,3.79,302956.59
50,2021-03-09,301651.43,1705,17.05,303339.38
49,2021-03-07,301987.76,-333,3.33,301651.43
48,2021-03-06,300755.21,1245,12.45,301987.76
47,2021-03-05,299892.92,871,8.71,300755.21


## 4.1 Income Tax 

- Separar por ano
- Separar por day trade ou nao
- separar por ação, opção e mercado futuro
- Separar por % de imposto

In [27]:
### Tax Rules
# each trade = 1% withholding tax if profit > 0 
# income tax is calculated by month = 20% * profit - withholdtax if it´s a day trade, normal trade 15%, if loss no tax

df1['withholding_tax'] = df1['Lucro'].apply(lambda x: abs(0.01*x) if x > 0 else 0 )

df1['Profit_minus_withholding_tax'] = df1.apply (lambda x: x['Lucro'] - x['withholding_tax'] if x['Lucro'] > 0 else x['Lucro'], axis =1 )



aux_tax_0 =  df1.groupby(['year','month', 'Market', 'day_trade']).agg( Profit= ('Lucro', 'sum'),Withholding_tax=('withholding_tax','sum'),Deals=('Ativo','count'),Profit_minus_tax= ('Profit_minus_withholding_tax', 'sum')).reset_index()


aux_tax_0['income_tax'] = aux_tax_0.apply(lambda x: 0.20 * x['Profit_minus_tax'] if (x['day_trade'] == 1) & (x['Profit_minus_tax'] > 0) else 0.15* x['Profit_minus_tax'] if   (x['day_trade'] == 0 )& (x['Profit_minus_tax'] > 0)   else 0, axis=1 )
aux_tax_0['Profit_net'] = aux_tax_0.apply(lambda x: x['Profit_minus_tax'] - x['income_tax'], axis=1 )


aux_tx_summary = aux_tax_0.groupby(['year','month', 'day_trade']).agg(Deals=('Deals','count'),Profit= ('Profit', 'sum'),Withholding_tax=('Withholding_tax','sum'), Income_tax = ('income_tax', 'sum'), Profit_net = ('Profit_net', 'sum') ).sort_values('month')

aux_tx_by_mkt = aux_tax_0.groupby(['year','month', 'Market','day_trade']).agg(Deals=('Deals','count'),Profit= ('Profit', 'sum'),Withholding_tax=('Withholding_tax','sum'), Income_tax = ('income_tax', 'sum'), Profit_net = ('Profit_net', 'sum') ).sort_values('month')
#aux_tx_summary.to_excel('../reports/tx_per_month-updated{}.xlsx'.format(y))
#aux_tx_by_mkt.to_excel('../reports/tx_per_mkt-updated{}.xlsx'.format(y))

In [28]:
##aux_tax = df1.groupby(['year','month', 'Market', 'day_trade']).agg( Deals=('Market','count'),Profit_Loss=('Lucro','sum')
##                                            ).sort_values('month').reset_index()
##
##
### Income Tax payable = 20% if day trade, 15% if not, 0 if loss
##aux_tax['Income_tax']= aux_tax.apply(lambda x: 0.20 * x['Profit_Loss'] if (x['day_trade'] == 1 )& (x['Profit_Loss'] > 0) else 
##                                       0.15* x['Profit_Loss'] if (x['Profit_Loss'] > 0) & (x['day_trade'] == 0 )   else 0, axis=1 )
##
## 
##
##
##aux_tax_per_month = aux_tax.groupby(['year','month', 'day_trade']).agg( Deals=('Deals','sum'),Profit_Loss=('Profit_Loss','sum'), Tax_income= ('Income_tax', 'sum')
##                                            ).sort_values('month')
##
##
##aux_tx_all_summary = aux_tax.groupby(['year','month','Market', 'day_trade']).agg( Deals=('Deals','sum'),Profit_Loss=('Profit_Loss','sum'), Tax_income= ('Income_tax', 'sum')
##                                            ).sort_values('month')
##
###aux_tax_per_month.to_excel('reports/income_tx_per_month-updated{}.xlsx'.format(y))

In [29]:
aux_tx_summary


Deals  Profit  Withholding_tax  Income_tax  Profit_net
year month day_trade                                                        
2021 2     0              3    4500            45.00     668.250    3786.750
           1              3    5938            72.62    1173.076    4692.304
     3     0              3     937            20.40     162.840     753.760
           1              3   16314           177.09    3227.382   12909.528
2020 10    1              1  105197          1128.45   20813.710   83254.840
     11    1              1  111954          1214.95   22147.810   88591.240
     12    1              1   57995           644.21   11470.158   45880.632

# 5.0  Send email

In [30]:
emails = pd.read_excel("../email.xls")
name = [ n for n in emails.Name]
email = [ e for e in emails.email]


##for i in range (len(emails)):
##    nom = name[i]
##    email_to = email[i]
##    
##    enviar_email(email, email_to, nom, yesterday_report, y, arquivo)
##    

In [34]:
#arquivo = ['../reports/cashflow2021_03_28.xls' ,'../reports/income_tx_per_month-updated2021_03_28.xlsx']

emails = pd.read_excel("../email.xls")
names = [ n for n in emails.Name]
emails = [ e for e in emails.email]


for i in range (len(emails)): 
    arquivo = ['../reports/cashflow2021_03_28.xlsx' ,'../reports/tx_per_month-updated2021_03_28.xlsx']
    
    #print(arquivos)
    

    send_mail_with_excel('../email_senha.txt' , names[i], emails[i],  'financial report', files(arquivo, names[i]) )


Add another file to Taiz´s report ( only excel, please!)? ( N to exit )n
Email sent to taizralves@gmail.com
Add another file to Augusto´s report ( only excel, please!)? ( N to exit )n
Email sent to hlima.augusto@gmail.com
